In [1]:
!pip install pgmpy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
from pgmpy.models import BayesianModel
from urllib.request import urlopen

In [6]:
names = "A,B,C,D,E,F,G,H,I,J,K,L,M,RESULT"
names = names.split(",")

In [7]:
data = pd.read_csv(urlopen("http://bit.do/heart-disease"), names=names)
data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,RESULT
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [8]:
model = BayesianModel([("A","B"),("B","C"),("C","RESULT")])
model.fit(data)

In [20]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)
q=infer.query(variables=["RESULT"], evidence={"A":22})

C:\Users\Sohil\AppData\Roaming\Python\Python39\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:518: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warn(

Finding Elimination Order: :   0%|                                                               | 0/2 [00:00<?, ?it/s]

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Eliminating: C:   0%|                                                                            | 0/2 [00:00<?, ?it/s]

Eliminating: B: 100%|███████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 666.29it/s]


In [25]:
print(q)

+-----------+---------------+
| RESULT    |   phi(RESULT) |
+===========+===============+
| RESULT(0) |        0.5387 |
+-----------+---------------+
| RESULT(1) |        0.1824 |
+-----------+---------------+
| RESULT(2) |        0.1196 |
+-----------+---------------+
| RESULT(3) |        0.1161 |
+-----------+---------------+
| RESULT(4) |        0.0433 |
+-----------+---------------+


In [26]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

In [27]:
bayesNet = BayesianModel()
bayesNet.add_node("M")
bayesNet.add_node("U")
bayesNet.add_node("R")
bayesNet.add_node("B")
bayesNet.add_node("S")

bayesNet.add_edge("M", "R")
bayesNet.add_edge("U", "R")
bayesNet.add_edge("B", "R")
bayesNet.add_edge("B", "S")
bayesNet.add_edge("R", "S")

In [28]:
cpd_A = TabularCPD('M', 2, values=[[.95], [.05]])
cpd_U = TabularCPD('U', 2, values=[[.85], [.15]])
cpd_H = TabularCPD('B', 2, values=[[.90], [.10]])

cpd_S = TabularCPD('S', 2, values=[[0.98, .88, .95, .6], [.02, .12, .05, .40]],
                   evidence=['R', 'B'], evidence_card=[2, 2])

cpd_R = TabularCPD('R', 2,
                   values=[[0.96, .86, .94, .82, .24, .15, .10, .05], [.04, .14, .06, .18, .76, .85, .90, .95]],
                   evidence=['M', 'B', 'U'], evidence_card=[2, 2,2])
bayesNet.add_cpds(cpd_A, cpd_U, cpd_H, cpd_S, cpd_R)

Finding Elimination Order: : 100%|█████████████████████████████████████████████████████| 2/2 [37:37<00:00, 1128.90s/it]


In [29]:
bayesNet.check_model()
print("Model is correct.")

Model is correct.


In [30]:
solver = VariableElimination(bayesNet)

In [37]:
result = solver.query(variables=['R'])
print("RESULT:", result.values[1])




  0%|                                                                                            | 0/4 [00:00<?, ?it/s]


Finding Elimination Order: :   0%|                                                               | 0/4 [00:00<?, ?it/s]



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]



Eliminating: M:   0%|                                                                            | 0/4 [00:00<?, ?it/s]



Eliminating: S:   0%|                                                                            | 0/4 [00:00<?, ?it/s]



Eliminating: U:   0%|                                                                            | 0/4 [00:00<?, ?it/s]



Eliminating: B: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 533.13it/s]

RESULT: 0.09378000000000002


In [38]:
bayesNet.get_independencies()

(M _|_ U, B)
(M _|_ B | U)
(M _|_ U | B)
(M _|_ S | R, B)
(M _|_ S | R, U, B)
(U _|_ M, B)
(U _|_ B | M)
(U _|_ M | B)
(U _|_ S | R, B)
(U _|_ S | M, R, B)
(B _|_ M, U)
(B _|_ U | M)
(B _|_ M | U)
(S _|_ M, U | R, B)
(S _|_ U | M, R, B)
(S _|_ M | R, U, B)